# Install and import packages

Be sure to select GPU as hardward accelerator.

In [1]:
# Transformers installation
! pip install transformers
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

     |████████████████████████████████| 1.8MB 5.9MB/s 
     |████████████████████████████████| 2.9MB 54.0MB/s 
     |████████████████████████████████| 890kB 29.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b88fea6db45dd26ccabcb12f101e861437d1b668beaa238d5d2f68080e03f5be
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import os
import re 
import pickle
import numpy as np
import transformers

# Prepare i2b2 2010 data
Code taken from : https://github.com/EmilyAlsentzer/clinicalBERT/blob/master/downstream_tasks/i2b2_preprocessing/i2b2_2010_relations/Reformat.ipynb

In [8]:
# !ls drive/MyDrive/Crowdfunding/concept_assertion_relation_training_data/beth/concept| wc -l

73


In [9]:
# !ls drive/MyDrive/Crowdfunding/concept_assertion_relation_training_data/partners/concept| wc -l

97


In [33]:
# TRAIN_DIRS = [
#     '/content/drive/MyDrive/Crowdfunding/concept_assertion_relation_training_data/beth/',
#     '/content/drive/MyDrive/Crowdfunding/concept_assertion_relation_training_data/partners/',
# ]
# TEST_DIR = '/content/drive/MyDrive/Crowdfunding/reference_standard_for_test_data/'

In [34]:
# def process_concept(concept_str):
#     """
#     takes string like
#     'c="asymptomatic" 16:2 16:2||t="problem"'
#     and returns dictionary like
#     {'t': 'problem', 'start_line': 16, 'start_pos': 2, 'end_line': 16, 'end_pos': 2}
#     """
#     try:
#         position_bit, problem_bit = concept_str.split('||')
#         t = problem_bit[3:-1]
        
#         start_and_end_span = next(re.finditer('\s\d+:\d+\s\d+:\d+', concept_str)).span()
#         c = concept_str[3:start_and_end_span[0]-1]
#         c = [y for y in c.split(' ') if y.strip() != '']
#         c = ' '.join(c)

#         start_and_end = concept_str[start_and_end_span[0]+1 : start_and_end_span[1]]
#         start, end = start_and_end.split(' ')
#         start_line, start_pos = [int(x) for x in start.split(':')]
#         end_line, end_pos = [int(x) for x in end.split(':')]
        
#         # Stupid and hacky!!!! This particular example raised a bug in my code below.
# #         if c == 'folate' and start_line == 43 and start_pos == 3 and end_line == 43 and end_pos == 3:
# #             start_pos, end_pos = 2, 2
        
#     except:
#         print(concept_str)
#         raise
    
#     return {
#         't': t, 'start_line': start_line, 'start_pos': start_pos, 'end_line': end_line, 'end_pos': end_pos,
#         'c': c, 
#     }

# def build_label_vocab(base_dirs):
#     seen, label_vocab, label_vocab_size = set(['O']), {'O': 'O'}, 0
    
#     for base_dir in base_dirs:
#         concept_dir = os.path.join(base_dir, 'concept')

        
#         assert os.path.isdir(concept_dir), "Directory structure doesn't match!"

        
#         ids = set([x[:-4] for x in os.listdir(concept_dir) if x.endswith('.con')])

#         for i in ids:
#             with open(os.path.join(concept_dir, '%s.con' % i)) as f:
#                 concepts = [process_concept(x.strip()) for x in f.readlines()]
#             for c in concepts:
#                 if c['t'] not in seen:
#                     label_vocab_size += 1
#                     label_vocab['B-%s' % c['t']] = 'B-%s' % c['t'] # label_vocab_size
#                     label_vocab_size += 1
#                     label_vocab['I-%s' % c['t']] = 'I-%s' % c['t'] # label_vocab_size
#                     seen.update([c['t']])
#     return label_vocab, label_vocab_size

# def reformatter(base, label_vocab, txt_dir = None, concept_dir = None):
#     if txt_dir is None: txt_dir = os.path.join(base, 'txt')
#     if concept_dir is None: concept_dir = os.path.join(base, 'concept')
    
#     assert os.path.isdir(txt_dir) and os.path.isdir(concept_dir), "Directory structure doesn't match!"
    
#     txt_ids = set([x[:-4] for x in os.listdir(txt_dir) if x.endswith('.txt')])
#     concept_ids = set([x[:-4] for x in os.listdir(concept_dir) if x.endswith('.con')])
    
#     assert txt_ids == concept_ids, (
#         "id set doesn't match: txt - concept = %s, concept - txt = %s"
#         "" % (str(txt_ids - concept_ids), str(concept_ids - txt_ids))
#     )
    
#     ids = txt_ids
    
#     reprocessed_texts = {}
#     for i in ids:
#         with open(os.path.join(txt_dir, '%s.txt' % i), mode='r') as f:
#             lines = f.readlines()
#             txt = [[y for y in x.strip().split(' ') if y.strip() != ''] for x in lines]
#             line_starts_with_space = [x.startswith(' ') for x in lines]
#         with open(os.path.join(concept_dir, '%s.con' % i), mode='r') as f:
#             concepts = [process_concept(x.strip()) for x in f.readlines()]
            
#         labels = [['O' for _ in line] for line in txt]
#         for c in concepts:
#             if c['start_line'] == c['end_line']:
#                 line = c['start_line']-1
#                 p_modifier = -1 if line_starts_with_space[line] else 0
#                 text = (' '.join(txt[line][c['start_pos']+p_modifier:c['end_pos']+1+p_modifier])).lower()
#                 assert text == c['c'], (
#                     "Text mismatch! %s vs. %s (id: %s, line: %d)\nFull line: %s"
#                     "" % (c['c'], text, i, line, txt[line])
#                 )
                
#             for line in range(c['start_line']-1, c['end_line']):
#                 p_modifier = -1 if line_starts_with_space[line] else 0
#                 start_pos = c['start_pos']+p_modifier if line == c['start_line']-1 else 0
#                 end_pos   = c['end_pos']+1+p_modifier if line == c['end_line']-1 else len(txt[line])
                
#                 if line == c['end_line'] - 1: labels[line][end_pos-1] = label_vocab['I-%s' % c['t']]                
#                 if line == c['start_line'] - 1: labels[line][start_pos] = label_vocab['B-%s' % c['t']]
#                 for j in range(start_pos + 1, end_pos-1): labels[line][j] = label_vocab['I-%s' % c['t']]
            
#         joined_words_and_labels = [zip(txt_line, label_line) for txt_line, label_line in zip(txt, labels)]

#         out_str = '\n\n'.join(
#             ['\n'.join(['%s %s' % p for p in joined_line]) for joined_line in joined_words_and_labels]
#         )
        
#         reprocessed_texts[i] = out_str
        
#     return reprocessed_texts

In [35]:
# label_vocab, label_vocab_size = build_label_vocab([
#     '/content/drive/MyDrive/Crowdfunding/concept_assertion_relation_training_data/beth/',
#     '/content/drive/MyDrive/Crowdfunding/concept_assertion_relation_training_data/partners/',
# ])

In [36]:
# label_vocab

{'B-problem': 'B-problem',
 'B-test': 'B-test',
 'B-treatment': 'B-treatment',
 'I-problem': 'I-problem',
 'I-test': 'I-test',
 'I-treatment': 'I-treatment',
 'O': 'O'}

In [38]:
# reprocessed_texts = {
#     'beth':     reformatter('/content/drive/MyDrive/Crowdfunding/concept_assertion_relation_training_data/beth/', label_vocab),
#     'partners': reformatter('/content/drive/MyDrive/Crowdfunding/concept_assertion_relation_training_data/partners/', label_vocab),
#     'test':     reformatter(
#         '/content/drive/MyDrive/Crowdfunding/reference_standard_for_test_data/', label_vocab,
#         txt_dir='/content/drive/MyDrive/Crowdfunding/test_data/',
#         concept_dir='/content/drive/MyDrive/Crowdfunding/reference_standard_for_test_data/concepts'
#     ),
# }

In [39]:
# for key, txt_by_record in reprocessed_texts.items(): print("%s: %d" % (key, len(txt_by_record)))

beth: 73
partners: 97
test: 256


In [42]:
# print(reprocessed_texts['beth']['record-37'][200:400])


having O
No O
Known B-problem
Allergies I-problem
to I-problem
Drugs I-problem

Attending O
: O
Sarah O
Q. O
George O
, O
M.D. O

Chief O
Complaint O
: O

Hip B-problem
fracture I-problem

Major O
Su


In [43]:
# np.random.seed(1)

In [44]:
# all_partners_train_ids = np.random.permutation(list(reprocessed_texts['partners'].keys()))
# N = len(all_partners_train_ids)
# N_train = int(0.9 * N)

# partners_train_ids = all_partners_train_ids[:N_train]
# partners_dev_ids = all_partners_train_ids[N_train:]

In [45]:
# print("Partners # Patients: Train: %d, Dev: %d" %(len(partners_train_ids), len(partners_dev_ids)))

Partners # Patients: Train: 87, Dev: 10


In [46]:
# all_beth_train_ids = np.random.permutation(list(reprocessed_texts['beth'].keys()))
# N = len(all_beth_train_ids)
# N_train = int(0.9 * N)

# beth_train_ids = all_beth_train_ids[:N_train]
# beth_dev_ids = all_beth_train_ids[N_train:]

In [47]:
# print("Beth # Patients: Train: %d, Dev: %d" % (len(beth_train_ids), len(beth_dev_ids)))

Beth # Patients: Train: 65, Dev: 8


In [48]:
# print("Merged # Patients: Train: %d, Dev: %d" % (
#   len(partners_train_ids) + len(beth_train_ids), len(beth_dev_ids) + len(partners_dev_ids)
# ))

Merged # Patients: Train: 152, Dev: 18


In [49]:
# merged_train_txt = '\n\n'.join(np.random.permutation(
#     [reprocessed_texts['partners'][i] for i in partners_train_ids] + 
#     [reprocessed_texts['beth'][i] for i in beth_train_ids]
# ))
# merged_dev_txt = '\n\n'.join(np.random.permutation(
#     [reprocessed_texts['partners'][i] for i in partners_dev_ids] + 
#     [reprocessed_texts['beth'][i] for i in beth_dev_ids]
# ))
# merged_test_txt = '\n\n'.join(np.random.permutation(list(reprocessed_texts['test'].values())))

In [50]:
# print("Merged # Samples: Train: %d, Dev: %d, Test: %d" % (
#     len(merged_train_txt.split('\n\n')),
#     len(merged_dev_txt.split('\n\n')),
#     len(merged_test_txt.split('\n\n'))
# ))

Merged # Samples: Train: 14693, Dev: 1622, Test: 27626


In [51]:
# partners_train_txt = '\n\n'.join(np.random.permutation(
#     [reprocessed_texts['partners'][i] for i in partners_train_ids]
# ))
# partners_dev_txt = '\n\n'.join(np.random.permutation(
#     [reprocessed_texts['partners'][i] for i in partners_dev_ids]
# ))
# partners_test_txt = '\n\n'.join(np.random.permutation(list(reprocessed_texts['test'].values())))

In [54]:
# OUT_FILES = {
#     'merged_train': '/content/drive/MyDrive/Crowdfunding/i2b2_2010_processed/merged/train.tsv',
#     'merged_dev':   '/content/drive/MyDrive/Crowdfunding/i2b2_2010_processed/merged/dev.tsv',
#     'merged_test':  '/content/drive/MyDrive/Crowdfunding/i2b2_2010_processed/merged/test.tsv', 
#     'partners_train': '/content/drive/MyDrive/Crowdfunding/i2b2_2010_processed/partners/train.tsv',
#     'partners_dev':   '/content/drive/MyDrive/Crowdfunding/i2b2_2010_processed/partners/dev.tsv',
#     'partners_test':  '/content/drive/MyDrive/Crowdfunding/i2b2_2010_processed/partners/test.tsv', 
#     'vocab': '/content/drive/MyDrive/Crowdfunding/i2b2_2010_processed/label_vocab.pkl'
# }

In [55]:
# with open(OUT_FILES['merged_train'], mode='w') as f: f.write(merged_train_txt)
# with open(OUT_FILES['merged_dev'], mode='w') as f: f.write(merged_dev_txt)
# with open(OUT_FILES['merged_test'], mode='w') as f: f.write(merged_test_txt)
# with open(OUT_FILES['partners_train'], mode='w') as f: f.write(partners_train_txt)
# with open(OUT_FILES['partners_dev'], mode='w') as f: f.write(partners_dev_txt)
# with open(OUT_FILES['partners_test'], mode='w') as f: f.write(partners_test_txt)
# with open(OUT_FILES['vocab'], mode='wb') as f: pickle.dump(label_vocab, f)

# Import processed i2b2 data

Code adapted from: https://github.com/EmilyAlsentzer/clinicalBERT/blob/master/downstream_tasks/run_ner.py



In [3]:
def LoadData(path):
  with open(path) as f:
    lines = []
    words = []
    labels = []
    for line in f:
        line = line.strip()
        if len(line) == 0: #i.e. we're in between sentences
            assert len(words) == len(labels)
            if len(words) == 0:
                continue
            lines.append([words, labels])
            words = []
            labels = []
            continue
        
        word = line.split()[0]
        label = line.split()[-1]
        words.append(word)
        labels.append(label)
  return lines

In [4]:
def ParseData(lines):
  output_texts = []
  output_labels = []
  for (i, line) in enumerate(lines):
    words,labels = line
    words = [w for w in words] 
    labels = [l for l in labels]
    output_texts.append(words)
    output_labels.append(labels)
  return output_texts,output_labels

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [154]:
train = LoadData("/content/drive/MyDrive/Crowdfunding/i2b2_2010_processed/merged/train.tsv")
dev = LoadData("/content/drive/MyDrive/Crowdfunding/i2b2_2010_processed/merged/dev.tsv")
test = LoadData("/content/drive/MyDrive/Crowdfunding/i2b2_2010_processed/merged/test.tsv")

In [155]:
train_texts, train_labels = ParseData(train)
val_texts, val_labels = ParseData(dev)
test_texts, test_labels = ParseData(test)

# Prepare data for BERT

Next, let's create encodings for our tokens and tags. For the tags, we can start by just create a simple mapping which
we'll use in a moment:

In [156]:
tags = train_labels + val_labels + test_labels

In [157]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [158]:
id2tag

{0: 'I-problem',
 1: 'B-problem',
 2: 'B-test',
 3: 'B-treatment',
 4: 'O',
 5: 'I-test',
 6: 'I-treatment'}

To encode the tokens, we'll use a pre-trained DistilBert tokenizer. We can tell the tokenizer that we're dealing with
ready-split tokens rather than full sentence strings by passing `is_split_into_words=True`. We'll also pass
`padding=True` and `truncation=True` to pad the sequences to be the same length. Lastly, we can tell the model to
return information about the tokens which are split by the wordpiece tokenization process, which we will need in a
moment.

In [153]:
from transformers import AutoTokenizer
#tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
#tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_Discharge_Summary_BERT")
#tokenizer = AutoTokenizer.from_pretrained("bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [159]:
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

Great, so now our tokens are nicely encoded in the format that they need to be in to feed them into our DistilBert
model below.

Now we arrive at a common obstacle with using pre-trained models for token-level classification: many of the tokens in
the W-NUT corpus are not in DistilBert's vocabulary. Bert and many models like it use a method called WordPiece
Tokenization, meaning that single words are split into multiple tokens such that each token is likely to be in the
vocabulary. For example, DistilBert's tokenizer would split the Twitter handle `@huggingface` into the tokens `['@',
'hugging', '##face']`. This is a problem for us because we have exactly one tag per token. If the tokenizer splits a
token into multiple sub-tokens, then we will end up with a mismatch between our tokens and our labels.

One way to handle this is to only train on the tag labels for the first subtoken of a split token. We can do this in 🤗
Transformers by setting the labels we wish to ignore to `-100`. In the example above, if the label for
`@HuggingFace` is `3` (indexing `B-corporation`), we would set the labels of `['@', 'hugging', '##face']` to
`[3, -100, -100]`.

Let's write a function to do this. This is where we will use the `offset_mapping` from the tokenizer as mentioned
above. For each sub-token returned by the tokenizer, the offset mapping gives us a tuple indicating the sub-token's
start position and end position relative to the original token it was split from. That means that if the first position
in the tuple is anything other than `0`, we will set its corresponding label to `-100`. While we're at it, we can
also set labels to `-100` if the second position of the offset mapping is `0`, since this means it must be a
special token like `[PAD]` or `[CLS]`.

> **NOTE:** Due to a recently fixed bug, -1 must be used instead of -100 when using TensorFlow in 🤗 Transformers <= 3.02.

In [160]:
def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_labels, train_encodings)
val_labels = encode_tags(val_labels, val_encodings)

The hard part is now done. Just as in the sequence classification example above, we can create a dataset object:

In [161]:
## PYTORCH CODE
import torch

class I2B2Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = I2B2Dataset(train_encodings, train_labels)
val_dataset = I2B2Dataset(val_encodings, val_labels)

In [162]:
len(train_labels)

14692

Now load in a token classification model and specify the number of labels:

In [163]:
## PYTORCH CODE
from transformers import BertForTokenClassification
#model = BertForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=len(unique_tags))
#model = BertForTokenClassification.from_pretrained("emilyalsentzer/Bio_Discharge_Summary_BERT", num_labels=len(unique_tags))
#model = BertForTokenClassification.from_pretrained("bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12", num_labels=len(unique_tags))
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(unique_tags))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

# Fine tune

In [164]:
## PYTORCH CODE
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=50,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Step,Training Loss
10,1.714600
20,1.149900
30,0.771500
40,0.738000
50,0.669400
60,0.623400
70,0.482600
80,0.430500
90,0.500900
100,0.372600


TrainOutput(global_step=2757, training_loss=0.13369523899512833, metrics={'train_runtime': 1289.4462, 'train_samples_per_second': 2.138, 'total_flos': 7862836835960928, 'epoch': 3.0})

In [165]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_loss': 0.18573081493377686,
 'eval_runtime': 12.3315,
 'eval_samples_per_second': 131.452}

In [166]:
output_dir = '/content/drive/MyDrive/Crowdfunding/BERT/'
model_to_save = model.module if hasattr(model, 'module') else model 
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/Crowdfunding/BERT/tokenizer_config.json',
 '/content/drive/MyDrive/Crowdfunding/BERT/special_tokens_map.json',
 '/content/drive/MyDrive/Crowdfunding/BERT/vocab.txt',
 '/content/drive/MyDrive/Crowdfunding/BERT/added_tokens.json')

# Run NER

In [91]:
import json
from transformers import pipeline
import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer
import itertools

In [169]:
# move model around, ultimately to CPU because that's where the dataset is located
# device = "cuda:0"
# model = model.to(device)
model = model.cpu()

In [170]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [49]:
#import data
p = []
for line in open('/content/drive/MyDrive/Crowdfunding/elmo_data_rounds_3-6.json', 'r'):
    p.append(json.loads(line))

In [51]:
p = p[0]

In [32]:
id2tag

{0: 'I-problem',
 1: 'B-problem',
 2: 'B-test',
 3: 'B-treatment',
 4: 'O',
 5: 'I-test',
 6: 'I-treatment'}

In [38]:
def FormatEntity(l, dic):
  r = []
  for token in l:
    ent = token['entity']
    id = int(ent.split('_')[1])
    iob = dic[id]
    token['entity'] = iob
    r.append(token)
  return r

In [171]:
#TAKES A FEW MINUTES--PATIENCE, YOUNG ONE
ner = []
for i in range(len(p)):
  campaign = {'words':[],'ents':[]}
  for sent in p[i]['lol']:
    sentence = TreebankWordDetokenizer().detokenize(sent)
    #run ner
    results = nlp(sentence)
    #reformat entity labels
    results_iob = FormatEntity(results, id2tag)
    #get lists for words and tokens
    words = []
    ents = []
    for n in range(len(results_iob)):
      words.append(results_iob[n]['word'])
      ents.append(results_iob[n]['entity'])
    campaign['words'].append(words)
    campaign['ents'].append(ents)
  #join lists of words and ents
  campaign['words'] = list(itertools.chain.from_iterable(campaign['words']))
  campaign['ents'] = list(itertools.chain.from_iterable(campaign['ents']))
  ner.append(campaign)

In [172]:
with open('/content/drive/MyDrive/Crowdfunding/BERT_ner_rounds_3-6.json', 'w') as fout:
    json.dump(ner , fout)